*italicized text*# Lab 1 - Introduction to optimization and operators

In [ ]:
#!python -m pip install "sepPlot @ git+http://zapad.stanford.edu/bob/pySepPlot.git@bd033869d29e5202212d81a285c27bfdc5907780"

In [ ]:
#!wget https://raw.githubusercontent.com/rgclapp007/GEE-lab1/main/bay.H

## Abstract

Here we will look at the general formulation of a linear optimization problem. We will define the model space, data space and the mapping matrix 
and focus on the properties of such a matrix. Then, we will look at operators and how to analyze them. Finally, we will apply a few roughening
operators to the topography map of the Bay area and interpret and analyze both the results and the operators.


## Linear optimization
In any scientific experiment, we try to measure a certain property or attribute. If we are lucky, that attribute can be measured directly.
Unfortunately, this is rarely the case. In general, we end up measuring another attribute that is not exactly what we want, but is related to
it by physical equations. Going from the attribute that we want to the attribute that we measure is considered the forward problem. The goal of
optimization is to go in the inverse direction, i.e. to obtain (or estimate) the model what we want from the data that we measure, by means of
solving physical equations.

Before we can invert our measurements, we first need to formulate the forward problem. In the framework of optimization, we define the
*model space* as the set of physical attributes that we want, and the *data space* as the physical attributes that we measure.
Everything else, including the physical equations, acquisition parameters or any auxiliary parameters, will be included in the mapping 
matrix. Once we defined these components, we can write any linear forward process as follows:

\begin{equation*}
d = F m,
\end{equation*}

where the vector $\mathbf d$ represents the data, the vector $\mathbf m$ represents the model, and the matrix $\mathbf F$ represents the
corresponding mapping operator. Equation 1 is only valid if the data space is linearly related to the model space, i.e. if the matrix
$\mathbf F$ is independent of the model vector $\mathbf m$. Notice that the size of the matrix $\mathbf F$ is determined by the size of the model
and data spaces. For instance, if the size of the model space is ${\rm Nm} \times 1$ and the size of data space is ${\rm Nd} \times 1$, then the
size of the mapping matrix is ${\rm Nd} \times {\rm Nm}$. Keep in mind that the model space and the data space are always portrayed as vectors of
size ${\rm N} \times 1$, regardless of the physical dimensionality.

The mapping matrix $\mathbf F$ converts a model vector into a data vector, in a process denominated *forward modeling*. We can get some
insight of such a process by looking  at the size of such a
matrix and analyzing which elements are non-zero, without actually looking at those non-zero values. For instance, a tall-thin matrix means that
the data space is larger than the model space, hence mapping from the model to the data includes significant spraying or scattering. The
larger data space too implies that the data cannot be exactly fit because of the presence of a non-trivial data null space (Aster,2013).
On the other hand, a short-wide matrix means that data space is smaller than the model space, hence mapping from the model to the data includes
significant summing or stacking. The larger model space too implies that several models can fit the data (Aster,2013). If the matrix is diagonal,
then each data point is only a scaled version of one corresponding model point, which is called scaling. If the matrix has non-zero values in
non-diagonal elements, then each data point is a linear combination of several model points, which is called filtering. If each column is the same
as the previous column but shifted down by one element, the process is considered stationary. In the worst-case scenario, all the elements are non-zero,
but this situation rarely occurs.



## Operators
The most direct way to perform forward modeling is saving matrix $\mathbf F$ and applying matrix-vector multiplication. However, in most of geophysical
problems the matrix is very sparse, i.e. most of its elements are zeros. Moreover, many processes are stationary, i.e., the same coefficients are
repeated over and over in every column. Therefore, instead of saving the whole matrix we can implement a procedure that loops through the elements
of the model and the data, utilizing only the non-zero matrix elements on the fly. This efficient implementation makes use of an *operator* that
corresponds to the matrix. In simple cases, each operator loop is similar to applying one row (or column) of the matrix. Notice that neither do we save
the elements of the matrix, nor compute any unnecessary elements. Also, since operators do not use matrix-vector multiplication, we do not have to convert
the data and model to vectors.

However, since we do not construct matrix ${\mathbf F}$ directly, it is often difficult to understand the behavior of the corresponding operator by just
looking at the code, especially for complicated operators. The solution isto find the operator filter's response, also called point-spread function (PSF).
The filter's response is computed by applying the operator on a model consisting on zeros and a single spike at one location. The output corresponds to
the column of the matrix $\mathbf F$ of that location. If the operator is a scalar, the output should have values at one location only. If the operator is
a filter, the output should have values at more than one location. If the operator is stationary, the filter response would be the same for all locations
of the spike (the model edges might be an exception).

## Your assignment

You will be working with a dataset that shows elevation arround the bay area.  We  will begin by reading in the dataset into a SepVector object. We begin by importing the SepVector and genericIO modules. We then read in the data into a sepVector object.

In [ ]:
import sepPython.modes        #Import SEP python module
io=sepPython.modes.defaultIO  #Get default IO that expects SEPlib datasets and uses sepVectors

bay=io.vectorFromStorage("./bay.H")
b=bay.getNdArray()

We can view the data by using the sepPlot module's Grey function.  We mark the cell to make an interative graphics cell using "%matplotlib ipympl". We import the matplotlib module which handles python graphics. We pass the **plt** object along with our **sepVector** object to Grey function. We've passed two additional arguments to display the labels.

In [ ]:
from sepPlot import Grey

grey=Grey(bay,label1="N-S",label2="E-W",width=500,height=700)
grey.image()

Next we will apply a series of operators to the image to see if we can gain new insignts into our map.  Each operator will inherit from a python base class that we will be using throughout the quarter.  Our first operator will apply a derivative in the North-South direction.

In [ ]:
from genericSolver.pyOperator import Operator
from genericSolver import pyVector
from numba import jit

class deriv1(Operator):
    """A simple operator that applies a derivative along the fast axis"""
    def __init__(self,domain):
        """Initialization an operator """
        
        if not isinstance(domain,pyVector.vector): 
            raise Exception("Expecting a python vector")
        
        #Store that vector space of the domain and range
        super().__init__(domain,domain)
        
    def forward(self,add,model,data):
        """Apply the operator
           add - boolean whether or not add to the data vector or zero it first
           model - model space (domain)
           data  - data space (range)"""
        
        #Make sure the model and data match the vectorspace the operator was initialized with
        self.checkDomainRange(model,data)
        
        #Zero the data if add -false
        if not add:
            data.zero()
        
        forward2D_1(model.getNdArray(),data.getNdArray())

        
    def adjoint(self,add,model,data):
        """Apply the adjoint of the operator
           add - boolean whether or not add to the data vector or zero it first
           model - model space (domain)
           data  - data space (range)"""
        
        raise Exception("Not implemented for now")

        
@jit(nopython=True)
def forward2D_1(model, data):
    for i2 in range(model.shape[0]):
        for i in range(1, model.shape[1]):
            data[i2,i] += model[i2,i] - model[i2,i - 1]

We can the apply our new operator to the view of the bay and look at the result.

In [ ]:
#make a copy of the bay vector
out=bay.clone()

#create a version of the operaor
op=deriv1(bay)

#Apply the operator
op.forward(False,bay,out)

#View the result
outP=Grey(out,label1="N-S",label2="E-W",width=500,height=700)
outP.image()

## Questions (part 1)

1. Write a python-code for a matrix-vector multiply using loops.

2.  How do we know if a matrix or operator is linear? Please recall your Linear Algebra course!

3. Given a simple line fitting problem where you have the values $\mathbf y = \{y1, y2, y3\}$ and the coordinates $\mathbf x = \{x1, x2, x3\}$, and
we want to fit a line with slope $a$ and y-intercept $b$ such as $\mathbf y = a \mathbf x + b$

  a. What is the data space, model space and the mapping matrix (write out all the elements)?

  b. Given that we are fitting a line, is $\mathbf y$ linearly related to $\mathbf x$? Show your proof.

4. In the relationship $\mathbf y = a \mathbf x^2$, is $\mathbf y$ linearly related to $\mathbf x$? If not, how can you reformulate it to make it so?

5.  Assume that the data is three dimensional $d(x_1, x_2, t)$ and model is two dimensional $m(y_1, y_2)$:

   a. How do you convert the model and data to the form $\mathbf d = \mathbf F \mathbf m$?

   b. What would the sizes of $\mathbf d$, $\mathbf F$ and $\mathbf m$ be? Use a prefix $\rm N$ to indicate size, .e.g. $\rm Nx_1$.

6. Assume that the data is one dimensional $d(x_1)$ and model is also one dimensional $m(x_1)$ and the operation we want to apply is first-order
backward finite difference (look it up if you do not already know it). Show the computational cost (size of memory required, number of summations and
multiplications) of doing a matrix-vector multiply.

7. Redo the same previous question but with operators instead of matrix-vector multiply.



In [ ]:
# 1. Write a python-code for a matrix-vector multiply using loops.
import numpy as np

def matVecMult(v, m, y, add = False):
    
    if m.shape[0] != y.shape[0]:
        raise ValueError('m.shape[0] should be equal to y.shape[0]')
    if m.shape[1] != v.shape[0]:
        raise ValueError('m.shape[1] should be equal to v.shape[0]') 
    
    if not add:
        y[:] = 0.0
    
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            y[i] += m[i][j] * v[j]
                    
d1 = 10
d2 = 20
m = np.ones((d2, d1))
v = np.ones(d1)
y = np.ones(d2)

matVecMult(v, m, y, add = False)
print(y)

2. If the matrix or operator is not related to the model, then it is linear. Otherwise, it is considered as a nonlinear problem since the matrix or operator includes the model parameters. Mathmatically, with $\mathbf d = \mathbf F (\mathbf m)$, we can test the following relationships: $ \alpha \mathbf d = \mathbf F (\alpha \mathbf m)$ and $ \alpha \mathbf d_1 + \beta \mathbf d_2  = \mathbf F (\alpha \mathbf m_1 + \beta  \mathbf m_2 ) = \alpha\mathbf F( \mathbf m_1) +  \beta  \mathbf F(\mathbf m_2 )$

3. In the form of $\mathbf d = \mathbf F \mathbf m$, this problem can be written as
$$
\left(\begin{array}{l}
y1  \\
y2  \\
y3
\end{array}\right) =
\left(\begin{array}{ll}
     x1 & 1 \\ 
     x2 & 1 \\
     x3 & 1
\end{array}\right)
\left(\begin{array}{l}
a \\
b
\end{array}\right)
$$
where the model space is $ m = [a, b]^{T} $, the data space is $ d = [y1, y2, y3]^T $, and the mapping matrix is 
$
F = \left(\begin{array}{llllll}
     x1 & 1 \\ 
     x2 & 1 \\
     x3 & 1
\end{array}\right)
$. \
$\mathbf y$ is not linearly related to $\mathbf x$. Given a new $x^* = \alpha x$ with $\alpha \neq 0$, there is $y^* = ax^* +b = a\alpha x + b \neq \alpha (ax +b)$. Therefore, $\mathbf y$ is not linearly related to $\mathbf x$

4. In the relationship $\mathbf y = a \mathbf x^2$,  $\mathbf y$ is not linearly related to $\mathbf x$. To make it so, I would like to assign $\mathbf x^2$ to a new variable such as $x^* = \mathbf x^2$. Thus, 
in the new relationship $\mathbf y = a x^*$,  $\mathbf y$ is linearly related to $x^*$, with $x^* = \mathbf x^2$.

5. I will first flatten both the data and model to be 1-D vectors. For example, the 1-D data vector is
$$
d^* = [d_{t_1}^{(1, 1)}, ..., d_{t_{Nt}}^{(1, 1)}, d_{t_1}^{(1, 2)}, ..., d_{t_{Nt}}^{(1, Nx_2)}, ..., d_{t_{Nt}}^{(Nx_1, Nx_2)}]^{T}
$$
the 1-D model vector is
$$
m^* = [m_{1, 1}, ..., m_{1, Ny_2}, m_{2, 1}, ..., m_{Ny_1, Ny_2}]
$$ \
The size of $\mathbf d $ will be $Nx_{1}  Nx_{2}  Nt$, the size of $\mathbf m $ will be $Ny_1  Ny_2$, and the size of $\mathbf F$ will be $Nx_{1}  Nx_{2}  Nt * Ny_1 Ny_2$.

6. In the matrix-vector multiply with the full matrix form, there will be $ (2 Nx_{1} + {Nx_{1}}^2)*SizePerDataType$ memory required, and there will be a total number of ${Nx_{1}}^2$ summations and ${Nx_{1}}^2$ multiplications.

7. In this case, there will be $ (2 Nx_{1})*SizePerDataType$ memory required, and there will be a total number of ${Nx_{1}}$ summations and no multiplication will be involved, if ignoring the first element (or last element).

## Part 2

Make two new classes that take a derivative in the E-W direction and one that applies a lapacian to the image. Define the classes in python and then apply them to the image of the bay.

In [ ]:
from genericSolver.pyOperator import Operator
from genericSolver import pyVector
from numba import jit

class deriv_EW(Operator):
    """A simple operator that applies a derivative along the slow axis"""
    def __init__(self,domain):
        """Initialization an operator """
        
        if not isinstance(domain,pyVector.vector): 
            raise Exception("Expecting a python vector")
        
        #Store that vector space of the domain and range
        super().__init__(domain,domain)
        
    def forward(self,add,model,data):
        """Apply the operator
           add - boolean whether or not add to the data vector or zero it first
           model - model space (domain)
           data  - data space (range)"""
        
        #Make sure the model and data match the vectorspace the operator was initialized with
        self.checkDomainRange(model,data)
        
        #Zero the data if add -false
        if not add:
            data.zero()
        
        forward2D_EW(model.getNdArray(),data.getNdArray())

        
    def adjoint(self,add,model,data):
        """Apply the adjoint of the operator
           add - boolean whether or not add to the data vector or zero it first
           model - model space (domain)
           data  - data space (range)"""
        
        raise Exception("Not implemented for now")

        
@jit(nopython=True)
def forward2D_EW(model, data):
    for i2 in range(model.shape[0]):
        for i in range(1, model.shape[2]):
            data[i2,:, i] += model[i2,:,i] - model[i2, :, i - 1]

In [ ]:
#make a copy of the bay vector
out=bay.clone()

#create a version of the operaor
op=deriv_EW(bay)

#Apply the operator
op.forward(False,bay,out)

#View the result
outP=Grey(out,label1="N-S",label2="E-W",width=500,height=700)
outP.image()

In [ ]:
from genericSolver.pyOperator import Operator
from genericSolver import pyVector
from numba import jit

class deriv_lap(Operator):
    """A simple operator that applies a derivative along both axes"""
    def __init__(self,domain):
        """Initialization an operator """
        
        if not isinstance(domain,pyVector.vector): 
            raise Exception("Expecting a python vector")
        
        #Store that vector space of the domain and range
        super().__init__(domain,domain)
        
    def forward(self,add,model,data):
        """Apply the operator
           add - boolean whether or not add to the data vector or zero it first
           model - model space (domain)
           data  - data space (range)"""
        
        #Make sure the model and data match the vectorspace the operator was initialized with
        self.checkDomainRange(model,data)
        
        #Zero the data if add -false
        if not add:
            data.zero()
        
        forward2D_lap(model.getNdArray(),data.getNdArray())

        
    def adjoint(self,add,model,data):
        """Apply the adjoint of the operator
           add - boolean whether or not add to the data vector or zero it first
           model - model space (domain)
           data  - data space (range)"""
        
        raise Exception("Not implemented for now")

        
@jit(nopython=True)
def forward2D_lap(model, data):
    for i2 in range(model.shape[0]):
        for i in range(1, model.shape[1]-1):
            for j in range(1, model.shape[2]-1):
                data[i2,i,j] += 4 * model[i2,i,j] - model[i2,i-1,j] - model[i2,i+1,j] - model[i2,i,j-1] - model[i2,i,j+1]

In [ ]:
#make a copy of the bay vector
out=bay.clone()

#create a version of the operaor
op=deriv_lap(bay)

#Apply the operator
op.forward(False,bay,out)

#View the result
outP=Grey(out,label1="N-S",label2="E-W",width=500,height=700)
outP.image()

## Extra Lab (Wizards only)
In chapter 1 of GIEE you first learn about filtering.
Next you learn about linear interpolation.
This is an exercise where you try to put the two together.
I give you lots of hints, then we talk about it.
This is the first year this assignment is given,
so, I have little idea how difficult you will find it.
PLEASE LIMIT YOURSELF TO TWO HOURS.
Do not expect to finish.
Please turn it in on time.
We will discuss it in subsequent class periods until we all know the answer.

### Background
We want to have filters that change their impulse response with time.
The obvious code seems to require big memory with size being this product:
data length times maximum filter lag.
This isn't bad in one physical dimension, such as time, but is oppressive
in higher spatial dimensions, such as in $(t,x)$, or $(t,x,z)$.
But here, as warm up, we try to put it together in time alone.

Fortunately, applications calling for time (and space) variable filters
generally want the filter to change slowly with time.
Thus we envision parameterizing filters by a coarse mesh (widely spaced time points).
From this coarse mesh, linear interpolation finds the effective filter
at each point in time.

### Exercise
Write a python code,
to define time-variable convolution with a filter linearly interpolated from a coarse mesh.
Ignore the adjoint.
You are invited to discuss this problem with others before class.
LIMIT YOUR TIME ON THIS PROBLEM TO TWO HOURS.

## Hints

The equation for filtering data $x(t)$ with filter $b(\tau)$ is
\begin{equation}
y(t) \quad=\quad \int b(\tau)\, x(t-\tau)
\end{equation}
Practitioners will love it if we give them a filter changing with time:
\begin{equation}
\label{eqn:tvdecon0}
y(t) \quad=\quad \int b(\tau,t)\, x(t-\tau)
\end{equation}
The convolution part looks like

do t    {     
>do tau {
> > if operator itself    
> > >     y(t) +=  b(tau,t* x(t-tau)       



The declarations at the beginning of the code can
make it work like
like $\bf y = \bf B\bf x$ or
like $\bf y = \bf X\bf b$.


How do we think about $b(\tau,t)$?
It's evidently an array, but not really.  It's more like an outer product.
Using matrices, we might write it as
\begin{eqnarray}
b(\tau,t) \quad=\quad 
	\left[
		\begin{array}{c}
			\vdots \\
			\bf b_i \\
			\vdots
		\end{array}
	\right]
	\left[ \begin{array}{ccccc} 1.0 & .9 & .8 \cdots & 0.0 \end{array} \right]
\ +\ 
	\left[
		\begin{array}{c}
			\vdots \\
			\bf b_{i+1} \\
			\vdots
		\end{array}
	\right]
	\left[ \begin{array}{ccccc} .0 & .1 & .2 &\cdots & 1.0 \end{array} \right]
\end{eqnarray}
where the idea is that $\bf b_i$ represents the filter at the previous mesh level
and $\bf b_{i+1}$ at the next level.
Filter lag is on the vertical axis while time $t$ is on the horizontal.






In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def convTimeInvariant(signal, kernel, result, add = False):
    ''' Convolution of a time invariant kernel with a time series

    Parameters
    ----------
    signal : 1D numpy array
        The time series to convolve
    kernel : 1D numpy array
        The time invariant kernel
    result : 1D numpy array
        The output array with the same size as signal + len(kernel) - 1
    add : bool, optional
        If True, add the result to result, otherwise overwrite result
    '''

    # get and check the size of the input
    nt = signal.shape[0]
    ntau = kernel.shape[0]
    
    if result.shape[0] != nt + ntau - 1:
        raise ValueError('result has the wrong size')

    # clear y if not add
    if not add:
        result[:] = 0.

    # perform the convolution
    for it in range(nt):
        for itau in range(ntau):
            result[it + itau] += kernel[itau] * signal[it]


def convTimeVariant(signal, kernel, kernel_time, result, add = False):
    ''' Convolution of a time variant kernel with a time series. The kernel is 
        represented by a coarse grid in time, and a linear interpolation is used 
        to evaluate the kernel at the time of the convolution.

    Parameters
    ----------
    signal : 1D numpy array
        The time series to convolve
    kernel : 2D numpy array (ntau, nsam)
        The coarse grid of the kernel
    kernel_time : 1D numpy array (nsam)
        The time axis of the kernel
    result : 1D numpy array
        The output array with the same size as result + len(kernel) - 1
    add : bool, optional
        If True, add the result to result, otherwise overwrite result
    
    Notes
    -----
    Currently, the kernel_time contains the time grid of the kernel instead of the real time asxis
    
    '''

    # get and check the size of the input
    nt   = signal.shape[0]
    ntau = kernel.shape[0]
    nsam = kernel.shape[1]
    
    
    if result.shape[0] != nt + ntau - 1:
        raise ValueError('result has the wrong size')
        
    if kernel_time.shape[0] != nsam:
        raise ValueError('kernel_time has the wrong size')
    
    # clear y if not add
    if not add:
        result[:] = 0.

    
    for it in range(nt):        
        
        # locate the current time in the kernel time axis and choose the right kernel
        if it < kernel_time[0]:
            kernel_in_use = kernel[:, 0]
        
        elif it >= kernel_time[-1]:
            kernel_in_use = kernel[:, -1]
        
        else:
            it1 = np.where(kernel_time <= it)[0][-1]
            it2 = it1 + 1
            kernel_in_use = (kernel[:, it1] * (kernel_time[it2] - it) + kernel[:, it2] * (it - kernel_time[it1])) / (kernel_time[it2] - kernel_time[it1])        
        
        # perform the convolution using the kernel derived from the coarse grid in time with linear interpolation
        for itau in range(ntau):
            result[it + itau] += kernel_in_use[itau] * signal[it]

In [ ]:
# Signal
my_signal = np.array([20, 22, 21, 25, 23, 24, 20, 21, 22, 25, 26, 27])

# Moving average filter
my_filter = np.array([1/3, 1/3, 1/3])

# Result array
my_result = np.zeros(my_signal.shape[0] + my_filter.shape[0] - 1)

# Perform the convolution 
convTimeInvariant(my_signal, my_filter, my_result, add = False)

plt.plot(my_signal)
plt.plot(my_result)
plt.show()

In [ ]:
# Signal
my_signal = np.array([20, 22, 21, 25, 23, 24, 20, 21, 22, 25, 26, 27])

# Moving average filter
my_filter = np.array([[1/2, 1/4, 1/4], [1/3, 1/3, 1/3], [1/4, 1/4, 1/2], [1, 0, 0]]).T
my_filter_time = np.array([2, 3, 5, 12])

# Result array
my_result2 = np.zeros(my_signal.shape[0] + my_filter.shape[0] - 1)

# Perform the convolution 
convTimeVariant(my_signal, my_filter, my_filter_time, my_result2, add = False)

plt.plot(my_signal)
plt.plot(my_result)
plt.plot(my_result2, '--')
plt.show()
